In [2]:
c = 4
print('vocab size:%d' % (c+2))

vocab size:6


In [2]:
import gensim

In [8]:
model = gensim.models.KeyedVectors.load_word2vec_format(pre_embedding_zh, binary=False)

In [16]:
a = torch.Tensor([[1,2,3], [4,5,6]])

In [18]:
torch.mean(a, dim=0)

tensor([ 2.5000,  3.5000,  4.5000])

In [17]:
s = "we're friend"
s

"we're friend"

In [10]:
import nltk

In [18]:
nltk.word_tokenize(s)

['we', "'re", 'friend']

In [6]:
a = '我不喜欢你啊aaａ, Ｂａｃｋ，back'

In [2]:
import jieba

In [7]:
jieba.lcut(a)

['我',
 '不',
 '喜欢',
 '你',
 '啊',
 'aa',
 'ａ',
 ',',
 ' ',
 'Ｂ',
 'ａ',
 'ｃ',
 'ｋ',
 '，',
 'back']